In [47]:
import pandas as pd
import numpy as np

In [90]:
df = pd.read_csv('../../data/X2.csv')

In [91]:
df.fillna('', inplace=True)
df = df.applymap(lambda x: str(x).lower().strip())

pattern_string = '( - )|,|:|\(|\)'
preprocessed_titles = df["title"].replace(pattern_string, ' ', regex=True)

df.insert(len(df.columns), "preprocessed_titles", preprocessed_titles)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   instance_id          343 non-null    object
 1   brand                343 non-null    object
 2   cpu_brand            343 non-null    object
 3   cpu_model            343 non-null    object
 4   cpu_type             343 non-null    object
 5   cpu_frequency        343 non-null    object
 6   ram_capacity         343 non-null    object
 7   ram_type             343 non-null    object
 8   ram_frequency        343 non-null    object
 9   hdd_capacity         343 non-null    object
 10  ssd_capacity         343 non-null    object
 11  weight               343 non-null    object
 12  dimensions           343 non-null    object
 13  title                343 non-null    object
 14  preprocessed_titles  343 non-null    object
dtypes: object(15)
memory usage: 40.3+ KB


In [93]:
processors = ['core i3', 'core i5', 'core i7', 'intel', 'amd']
def check_processor(text):
    for p in processors:
        if(p in text):
            return p
    return 'unknown'        

In [94]:
df["all_cpu"] = df["cpu_brand"] + df["cpu_model"] + df["cpu_type"] + df["cpu_frequency"] + df["title"] + df["brand"]

df["all_cpu"]

0      intel. i5-3320mi5-3320mdual-core ( 2 core ). c...
1      1.6 ghz intel core i5-4200u. intel core i51.6 ...
2      1.6 ghz intel core i5. intel core i51.6 ghz in...
3      amazon.com : 15.6" hp 15-f009wm amd dual-core ...
4      1.7 ghz core i5-3317u. intel1.7 ghz core i5-33...
                             ...                        
338    intel core i5 ( 3rd gen ) 3320m / 2.6 ghz. int...
339    intel core i5 ( 3rd gen ) 3320m / 2.6 ghz. int...
340    intel core i5 ( 3rd gen ) 3320m / 2.6 ghz. int...
341    intel core i5 ( 3rd gen ) 3320m / 2.6 ghz. int...
342    intel core i7 ( 3rd gen ) 3520m / 2.9 ghz. int...
Name: all_cpu, Length: 343, dtype: object

In [95]:
df["blocking_processor"] = df["all_cpu"].apply(check_processor)
df["blocking_processor"]

0      core i5
1      core i5
2      core i5
3          amd
4      core i5
        ...   
338    core i5
339    core i5
340    core i5
341    core i5
342    core i7
Name: blocking_processor, Length: 343, dtype: object

In [100]:
grps = df["blocking_processor"].groupby(df["blocking_processor"])

In [101]:
grps.count()

blocking_processor
amd         16
core i3     39
core i5    197
core i7     79
intel       12
Name: blocking_processor, dtype: int64

In [119]:
def extract_brands(text):
    if(text):
        return text.split()[0]
    return 'unknown'

brands = set(df["brand"].apply(extract_brands).unique()) - set(['unknown'])

In [120]:
brands

{'acer', 'asus', 'dell', 'hp', 'lb1', 'lenovo'}

In [121]:
def get_blocking_brands(text):
    for b in brands:
        if(b in text):
            return b
    return 'unknown'

In [125]:
df["blocking_brand"] = df["title"].apply(get_blocking_brands)

In [126]:
grps = df["blocking_brand"].groupby(df["blocking_brand"])

In [127]:
grps.count()

blocking_brand
acer       56
asus        2
dell        9
hp         37
lb1         5
lenovo    234
Name: blocking_brand, dtype: int64

In [130]:
df["blocking_key"] = df["blocking_brand"] + ' ' + df["blocking_processor"]

In [131]:
df["blocking_key"].groupby(df["blocking_key"]).count()

blocking_key
acer amd            2
acer core i3       23
acer core i5       13
acer core i7        6
acer intel         12
asus core i5        2
dell amd            2
dell core i5        5
dell core i7        2
hp amd              6
hp core i3          4
hp core i5         25
hp core i7          2
lb1 core i3         5
lenovo amd          6
lenovo core i3      7
lenovo core i5    152
lenovo core i7     69
Name: blocking_key, dtype: int64